In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt  # importing module
face_cascade=cv2.CascadeClassifier("D:\\tanuj doument\\sublime text app\\haarcascade_frontalface_default.xml")  # to detect the frontal face
#eye_cascade=cv2.CascadeClassifier("D:\\tanuj doument\\sublime text app\\haarcascade_eye.xml")
#capturing images from video
cap=cv2.VideoCapture(0)   # to activate the webcam
skip=0
face_data=[]
face_section=[]    #variable declaration

while True:
    ret,frame=cap.read()  #return frame from the video
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    if ret==False:
        continue
    faces=face_cascade.detectMultiScale(gray_frame,1.3,5)  # to detect the face
    faces=sorted(faces,key=lambda f:f[2]*f[3],reverse=True)   # sorting face  in increasing order
    #eyes=eye_cascade.detectMultiScale(gray_frame,1.3,5)
    #print(faces)
    for face in faces:
        x,y,w,h=face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)  # to place rectangle on face
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset,:]
        face_section=cv2.resize(face_section,(100,100))
        skip+=1
        if skip%10==0:
            if face_section.size!=0:
               face_data.append(face_section)
            
    cv2.imshow("Video Frame",frame)
    cv2.imshow("frame_crop_image",np.array(face_section))
   # cv2.imshow("gray video frame",gray_frame)
    
    if cv2.waitKey(1)& 0xFF==ord('q'):
        break
dataset_path='D:/tanuj doument/sublime text app/'
face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))
file_name=input("enter person name")
np.save(dataset_path+file_name+'.npy',face_data)
cap.release()
cv2.destroyAllWindows()

enter person nameTanuj Kumar


In [2]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
face_reco=[]
dataset_path='D:/tanuj doument/sublime text app/'
import os
#data prep
class_id=0
labels=[]
name={}
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        name[class_id]=fx[:-4]
        data_item=np.load(dataset_path+fx)
        face_reco.append(data_item)
        #create labels
        target=class_id*np.ones(data_item.shape[0],).reshape(-1,1)
        class_id+=1
        labels.append(target)
        #np.concatenate((data_item,target),axis=1)
face_dataset=np.concatenate(face_reco,axis=0)
face_labels=np.concatenate(labels,axis=0)


In [5]:
df=pd.DataFrame(face_dataset,dtype=int)
col=[]
for i in range(30000):
    col.append(f'pixel{i}')
df.columns=col
df

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel29990,pixel29991,pixel29992,pixel29993,pixel29994,pixel29995,pixel29996,pixel29997,pixel29998,pixel29999
0,57,55,59,54,52,56,51,52,56,53,...,2,11,2,3,11,2,3,4,0,0
1,80,64,51,80,63,53,83,64,58,83,...,3,2,0,5,0,0,2,0,0,0
2,70,59,59,70,60,59,69,62,59,71,...,4,9,0,1,5,0,1,7,0,2
3,80,62,60,81,63,60,82,64,61,82,...,0,0,0,0,0,0,0,0,0,0
4,78,59,54,77,58,52,75,57,50,75,...,0,1,0,0,4,0,0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,201,210,223,201,210,223,201,210,223,202,...,223,201,211,223,201,210,223,199,209,221
277,200,207,219,203,210,223,203,210,224,202,...,218,199,210,218,199,209,218,199,209,217
278,200,209,223,201,210,224,202,212,224,204,...,221,201,207,220,202,209,222,202,210,220
279,204,211,222,206,212,224,204,212,225,203,...,219,195,206,218,192,205,217,191,206,215


In [6]:
from sklearn.svm import LinearSVC

In [10]:
svm_clf=LinearSVC(max_iter=1500)
svm_clf.fit(face_dataset,face_labels.flatten())

LinearSVC(max_iter=1500)

In [8]:
def transparentOverlay(src, overlay, pos=(0, 0), scale=1):
    overlay = cv2.resize(overlay, (0, 0), fx=scale, fy=scale)
    h, w, _ = overlay.shape  # Size of foreground
    rows, cols, _ = src.shape  # Size of background Image
    y, x = pos[0], pos[1]  # Position of foreground/overlay image
    # loop over all pixels and apply the blending equation
    for i in range(h):
        for j in range(w):
            if x + i >= rows or y + j >= cols:
                continue
            alpha = float(overlay[i][j][3] / 255.0)  # read the alpha channel
            src[x + i][y + j] = alpha * overlay[i][j][:3] + (1 - alpha) * src[x + i][y + j]
    return src

In [9]:

face_cascade=cv2.CascadeClassifier("D:\\tanuj doument\\sublime text app\\haarcascade_frontalface_default.xml")
specs_ori = cv2.imread("D:/tanuj doument/sublime text app/glass.png" , -1)
cigar_ori = cv2.imread("D:/tanuj doument/sublime text app/cigar.png" , -1)
mus_ori = cv2.imread("D:/tanuj doument/sublime text app/mustache.png" , -1)
filter_name=int(input("1.glasses and cigar \n2.cigar and mustache\n"))
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()  #return frame from the video
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    if ret==False:
        continue
    faces=face_cascade.detectMultiScale(gray_frame,1.3,5)
    faces=sorted(faces,key=lambda f:f[2]*f[3],reverse=True)
    #eyes=eye_cascade.detectMultiScal
    for face in faces:
        x,y,w,h=face
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset,:]
        face_section=cv2.resize(face_section,(100,100))
        out=svm_clf.predict(face_section.reshape(1,-1))
        cv2.putText(frame,name[int(out)],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA,False)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        glass_symin = int(y + 1.5 * h / 5)
        glass_symax = int(y + 2.5 * h / 5)
        sh_glass = glass_symax - glass_symin
            
        cigar_symin = int(y + 4 * h / 6)
        cigar_symax = int(y + 5.5 * h / 6)
        sh_cigar = cigar_symax - cigar_symin
            
        mus_symin = int(y + 3.5 * h / 6)
        mus_symax = int(y + 5 * h / 6)
        sh_mus = mus_symax - mus_symin
            
        face_glass_ori = frame[glass_symin:glass_symax, x:x+w]
        face_cigar_ori = frame[cigar_symin:cigar_symax, x:x+w]
        face_mus_ori = frame[mus_symin:mus_symax, x:x + w]
        specs = cv2.resize(specs_ori, (w, sh_glass),interpolation=cv2.INTER_CUBIC)
        cigar= cv2.resize(cigar_ori, (w, sh_cigar),interpolation=cv2.INTER_CUBIC)
        mus = cv2.resize(mus_ori, (w, sh_mus), interpolation=cv2.INTER_CUBIC)
        if(filter_name==1):
           transparentOverlay(face_glass_ori,specs)
        transparentOverlay(face_cigar_ori,cigar,(int(w/2),int(sh_cigar/2)))
        if(filter_name==2):
           transparentOverlay(face_mus_ori,mus)
        #print(int(out))
    cv2.imshow("Video Frame",frame)
   # cv2.imshow("frame_crop_image",face_section)
   # cv2.imshow("gray video frame",gray_frame)
    
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1.glasses and cigar 
2.cigar and mustache
1
